In [1]:
import os
from typing import *
import operator

from dotenv import load_dotenv
from numba.np.ufunc.parallel import build_gufunc_kernel

load_dotenv()
from IPython.display import Image, display
import sqlite3
from pydantic import BaseModel, Field


from langchain_ollama import ChatOllama
from langchain_together import ChatTogether
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage, RemoveMessage
from langchain_community.tools import TavilySearchResults
from langchain_community.document_loaders import WikipediaLoader
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph.errors import NodeInterrupt
from langgraph.types import interrupt, Command
from langgraph.constants import Send

In [2]:
# ------------------------------------------------------ #
# GLOBALS
# ------------------------------------------------------ #
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

analyst_instructions="""You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:

1. First, review the research topic:
{topic}

2. Examine any editorial feedback that has been optionally provided to guide creation of the analysts:

{human_analyst_feedback}

3. Determine the most interesting themes based upon documents and / or feedback above.

4. Pick the top {max_analysts} themes.

5. Assign one analyst to each theme."""

In [3]:
# ------------------------------------------------------ #
# MODELS
# ------------------------------------------------------ #
# chat_llm = ChatTogether(model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", api_key=TOGETHER_API_KEY)
chat_llm = ChatOllama(model='llama3.2:latest')

In [ ]:
# ------------------------------------------------------ #
# STATES
# ------------------------------------------------------ #
# --------------------------- #
# GENERATE ANALYSTS
# --------------------------- #
class Analyst(BaseModel):
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    description: str = Field(description="Description of the analyst focus, concerns, and motives.")
    @property
    def persona(self) -> str:
        return (f"Name: {self.name}\n"
                f"Role: {self.role}\n"
                f"Affiliation: {self.affiliation}\n"
                f"Description: {self.description}\n")


class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(description="Comprehensive list of analysts with their roles and affiliations.")


class GenerateAnalystsState(TypedDict):
    topic: str  # research topic
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[Analyst]

In [ ]:
# ------------------------------------------------------ #
# NODES
# ------------------------------------------------------ #
# --------------------------- #
# GENERATE ANALYSTS
# --------------------------- #

In [ ]:
# ------------------------------------------------------ #
# CONDITIONAL EDGES
# ------------------------------------------------------ #

In [ ]:
# ------------------------------------------------------ #
# WORKFLOWS
# ------------------------------------------------------ #
# --------------------------- #
# GENERATE ANALYSTS
# --------------------------- #

In [ ]:
# ------------------------------------------------------ #
# COMPILE
# ------------------------------------------------------ #
# --------------------------- #
# GENERATE ANALYSTS
# --------------------------- #

In [ ]:
# ------------------------------------------------------ #
# INVOCATIONS
# ------------------------------------------------------ #
# --------------------------- #
# GENERATE ANALYSTS
# --------------------------- #